# Primer notebook
En este primer notebook se dejara listo el proyecto para ser usado por yolo en la deteccion de imagenes, se utilizo de base el dataset coco que puede ser decargo de manera publica mediante el sigueinte link: https://www.kaggle.com/datasets/awsaf49/coco-2017-dataset
Para este proyecto se usara ya el dataste descrgado de manera local por ende no se descargara en este proyecto

Este notebook:
- Define rutas y constantes del proyecto
- Verifica que COCO está disponible en tu PC
- Define 3 clases objetivo: **car, airplane, truck**
- Define límites de imágenes para entrenar rápido
- Crea carpetas estándar para artefactos, modelos, salidas y dataset procesado


In [1]:
#Imports generales
from pathlib import Path
import os
import json
import random
from dataclasses import dataclass
from typing import Dict, Tuple
import numpy as np


1. Carpetas y rutas base 

In [2]:
# --- RUTAS DEL PROYECTO (WINDOWS + COCO LOCAL) ---
#Aqui se define las caretas y rutas que usarar el resto del proyecto para poder comunicasrse entre si, 
# ademas aclara en donde se encuentra el dataset, ademas de establecer la ruta de los modelos y del mlruns
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent

COCO_ROOT = Path(r"C:\Users\Johnny\Desktop\IA\data\archive\coco2017")
COCO_ANN_DIR = COCO_ROOT / "annotations"
COCO_TRAIN_IMG_DIR = COCO_ROOT / "train2017"
COCO_VAL_IMG_DIR = COCO_ROOT / "val2017"
COCO_TEST_IMG_DIR = COCO_ROOT / "test2017"

COCO_INSTANCES_TRAIN = COCO_ANN_DIR / "instances_train2017.json"
COCO_INSTANCES_VAL = COCO_ANN_DIR / "instances_val2017.json"

ARTIFACTS_DIR = PROJECT_ROOT / "artifacts"
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
DETECTION_DATASET_DIR = PROCESSED_DIR / "detection_dataset"
MODELS_DIR = PROJECT_ROOT / "models"
RUNS_DIR = PROJECT_ROOT / "runs"
MLRUNS_DIR = PROJECT_ROOT / "mlruns"

print("PROJECT_ROOT:", PROJECT_ROOT)
print("COCO_ROOT:", COCO_ROOT)


PROJECT_ROOT: c:\Users\Johnny\Desktop\IA
COCO_ROOT: C:\Users\Johnny\Desktop\IA\data\archive\coco2017


In [3]:
# --- FUNCIONES ÚTILES (CARPETAS Y VALIDACIONES) ---
#Creacion de las carpeytas previas
def ensure_dir(p: Path) -> None:
    p.mkdir(parents=True, exist_ok=True)

def assert_exists(p: Path, desc: str) -> None:
    if not p.exists():
        raise FileNotFoundError(f"Falta {desc}: {p}")

for d in [ARTIFACTS_DIR, PROCESSED_DIR, MODELS_DIR, RUNS_DIR]:
    ensure_dir(d)

print("Carpetas base listas.")


Carpetas base listas.


2. Configuracion del proyecto

In [4]:
# --- CONFIGURACIÓN DEL PROYECTO (CLASES + LÍMITES + PARÁMETROS) ---
#Dado que el dataset presenta un gran numero de imagenes se ha limitado 
#el numero de iagenes que se usaran, igual los batch y epochs por temas de tiempo y recursos
@dataclass
class ProjectConfig:
    target_classes: Tuple[str, str, str] = ("car", "airplane", "truck")
    max_images_per_class_train: int = 1200
    max_images_per_class_val: int = 250
    img_size: int = 640
    batch_size: int = 16
    epochs: int = 20
    seed: int = 42
    conf_threshold: float = 0.25
    iou_threshold: float = 0.5

CFG = ProjectConfig()
CFG


ProjectConfig(target_classes=('car', 'airplane', 'truck'), max_images_per_class_train=1200, max_images_per_class_val=250, img_size=640, batch_size=16, epochs=20, seed=42, conf_threshold=0.25, iou_threshold=0.5)

In [5]:
#para evitar la aleatoriedad en los resultados se genero lo siguiente 
# que cada vez que ejecutes el código, las operaciones aleatorias produzcan los mismos resultados, siempre que uses el mismo seed.
# --- REPRODUCIBILIDAD ---
def set_seed(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(CFG.seed)
print("Seed:", CFG.seed)


Seed: 42


3. Validaciones

In [6]:
#Verificacion de que las carepetas y archivos importantes si existan
# --- VERIFICACIÓN DE COCO LOCAL (SIN DESCARGAS) ---
assert_exists(COCO_ROOT, "la carpeta raíz de COCO")
assert_exists(COCO_ANN_DIR, "la carpeta annotations")
assert_exists(COCO_TRAIN_IMG_DIR, "la carpeta train2017")
assert_exists(COCO_INSTANCES_TRAIN, "instances_train2017.json")

if COCO_INSTANCES_VAL.exists():
    print("OK: instances_val2017.json encontrado.")
else:
    print("Aviso: instances_val2017.json NO está. Luego haremos split desde train en el Notebook 02.")

print("COCO local verificado.")


OK: instances_val2017.json encontrado.
COCO local verificado.


In [7]:
#Validaciones de COCO y clases que se usaran existan (truck car airplane)
# --- CARGA DE CATEGORÍAS Y VALIDACIÓN DE CLASES OBJETIVO ---
with open(COCO_INSTANCES_TRAIN, "r", encoding="utf-8") as f:
    coco_train = json.load(f)

categories = coco_train.get("categories", [])
cat_name_to_id = {c["name"]: c["id"] for c in categories}

missing = [c for c in CFG.target_classes if c not in cat_name_to_id]
if missing:
    raise ValueError(
        f"Estas clases no existen en COCO: {missing}\n"
        f"Ejemplos de clases disponibles: {list(cat_name_to_id.keys())[:25]}"
    )

print("Clases objetivo confirmadas:")
for name in CFG.target_classes:
    print("-", name, "-> category_id:", cat_name_to_id[name])


Clases objetivo confirmadas:
- car -> category_id: 3
- airplane -> category_id: 5
- truck -> category_id: 8


In [8]:
#bbox es una validacion que necesita yolo para el proceso de validacion
#indica la cantidad de objetos que hay en el dataset clasificados, por ejeplo 5135 obejtos con ID airplane
# --- ESTADÍSTICAS RÁPIDAS (CANTIDAD DE BBOX POR CLASE EN TRAIN) ---
target_cat_ids = {cat_name_to_id[n] for n in CFG.target_classes}
anns = coco_train.get("annotations", [])

id_to_name = {v: k for k, v in cat_name_to_id.items()}
bbox_counts = {name: 0 for name in CFG.target_classes}

for a in anns:
    cid = a.get("category_id")
    if cid in target_cat_ids:
        name = id_to_name[cid]
        if name in bbox_counts:
            bbox_counts[name] += 1

print("BBoxes (train) por clase:")
for k, v in bbox_counts.items():
    print("-", k, ":", f"{v:,}")


BBoxes (train) por clase:
- car : 43,867
- airplane : 5,135
- truck : 9,973


4. Conexiones con futuros notebooks

In [9]:
# --- ARCHIVOS ESTÁNDAR QUE USAREMOS EN LOS SIGUIENTES NOTEBOOKS ---
CLASS_MAP_PATH = ARTIFACTS_DIR / "class_map.json"
CONFIG_SNAPSHOT_PATH = ARTIFACTS_DIR / "config_snapshot.json"
DATASET_INDEX_DIR = ARTIFACTS_DIR / "dataset_index"

ensure_dir(DATASET_INDEX_DIR)

class_map = {i: name for i, name in enumerate(CFG.target_classes)}

with open(CLASS_MAP_PATH, "w", encoding="utf-8") as f:
    json.dump(class_map, f, indent=2)

with open(CONFIG_SNAPSHOT_PATH, "w", encoding="utf-8") as f:
    json.dump(CFG.__dict__, f, indent=2)

print("Guardado:")
print("-", CLASS_MAP_PATH)
print("-", CONFIG_SNAPSHOT_PATH)
print("-", DATASET_INDEX_DIR)


Guardado:
- c:\Users\Johnny\Desktop\IA\artifacts\class_map.json
- c:\Users\Johnny\Desktop\IA\artifacts\config_snapshot.json
- c:\Users\Johnny\Desktop\IA\artifacts\dataset_index


In [10]:
# --- MLflow (CONFIGURACIÓN LOCAL POR DEFECTO) ---
#Ese bloque configura MLflow para registrar tus experimentos de forma local, sin servidor remoto
MLFLOW_TRACKING_URI = f"file:{MLRUNS_DIR.as_posix()}"
MLFLOW_EXPERIMENT_NAME = "coco2017_car_airplane_truck"

print("MLFLOW_TRACKING_URI:", MLFLOW_TRACKING_URI)
print("MLFLOW_EXPERIMENT_NAME:", MLFLOW_EXPERIMENT_NAME)


MLFLOW_TRACKING_URI: file:c:/Users/Johnny/Desktop/IA/mlruns
MLFLOW_EXPERIMENT_NAME: coco2017_car_airplane_truck
